# Job Board Scraping Lab

In this lab you will first see a minimal but fully functional code snippet to scrape the LinkedIn Job Search webpage. You will then work on top of the example code and complete several chanllenges.

### Some Resources 

- [Requests library](http://docs.python-requests.org/en/master/#the-user-guide) documentation 
- [Beautiful Soup Doc](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Urllib](https://docs.python.org/3/library/urllib.html#module-urllib)
- [re lib](https://docs.python.org/3/library/re.html)
- [Scrapy](https://scrapy.org/)
- [List of HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)
- [HTML basics](http://www.simplehtmlguide.com/cheatsheet.php)
- [CSS basics](https://www.cssbasics.com/#page_start)

In [1]:
# Import the required libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

"""
This function searches job posts from LinkedIn and converts the results into a dataframe.
"""
def scrape_linkedin_job_search(keywords):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    # Assemble the full url with parameters
    scrape_url = ''.join([BASE_URL, 'keywords=', keywords])

    # Create a request to get the data from the server 
    page = requests.get(scrape_url)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for card in soup.select("div.result-card__contents"):
        title = card.findChild("h3", recursive=False)
        company = card.findChild("h4", recursive=False)
        location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
        titles.append(title.string)
        companies.append(company.string)
        locations.append(location.string)
    
    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    # Return dataframe
    return data

In [2]:
# Example to call the function

results = scrape_linkedin_job_search('data%20analysis')
results

,Title,Company,Location
0,IT Senior Associate,Kearney & Company,"McLean, VA, US"
1,Intern: Data Analysis/Process Improvement Prog...,Juniper Networks,"Sunnyvale, CA, US"
2,Associate / Business Analyst (Consulting Data ...,"Foresight Associates, LLC",Greater Minneapolis-St. Paul Area
3,"Software Engineer, Data Models & Analysis",Improbable,"Washington, D.C., DC, US"
4,"Manager, Reporting and Data Analysis",The Madison Square Garden Company,Greater New York City Area
5,Data Analysis Manager,Capital One,"McLean, VA, US"
6,Data Management and Analysis Services Analyst I,Conduent,"Ocoee, Florida"
7,"Manager, Data Analysis & Insights",The Coca-Cola Company,"Atlanta, GA, US"
8,Data Analyst,Viper Staffing Services L.L.C.,"Glendale, California, United States"
9,Sr. Bioinformatics Scientist,The EAC Agency,"Fremont, California, United States"


## Challenge 1

The first challenge for you is to update the `scrape_linkedin_job_search` function by adding a new parameter called `num_pages`. This will allow you to search more than 25 jobs with this function. Suggested steps:

1. Go to https://www.linkedin.com/jobs/search/?keywords=data%20analysis in your browser.
1. Scroll down the left panel and click the page 2 link. Look at how the URL changes and identify the page offset parameter.
1. Add `num_pages` as a new param to the `scrape_linkedin_job_search` function. Update the function code so that it uses a "for" loop to retrieve several pages of search results.
1. Test your new function by scraping 5 pages of the search results.

Hint: Prepare for the case where there are less than 5 pages of search results. Your function should be robust enough to **not** trigger errors. Simply skip making additional searches and return all results if the search already reaches the end.

In [69]:
# your code here

def scrape_linkedin_job_search_pages(keywords,num_pages):
    
    
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    scrape_url = ''.join([BASE_URL, 'keywords=', keywords])

    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)
    
    for num in range(num_pages):
        if num == 0:
            page = requests.get(scrape_url)
        else:
            page = requests.get(''.join([scrape_url,'&start=',str(num*25)]))
        soup = BeautifulSoup(page.text, 'html.parser')
        titles = []
        companies = []
        locations = []
        for card in soup.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
        zipped = zip(titles, companies, locations)
        for z in list(zipped):
            data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    return data


In [75]:
results = scrape_linkedin_job_search_pages('data%20analysis',6)
results

,Title,Company,Location
0,Sr. Data Analyst – Predictive & Inferential An...,Peyton Resource Group,"Arlington, Texas"
1,"Manager, Data and Analysis",Digitas North America,Greater New York City Area
2,Data Analysis Manager,Addison Group,"Houston, TX, US"
3,"Manager, Reporting and Data Analysis",The Madison Square Garden Company,Greater New York City Area
4,Senior Data Analyst,iTech Solutions,"Philadelphia, Pennsylvania"
...,...,...,...
145,Data Analyst,Brooksource,"Philadelphia, Pennsylvania"
146,Data Analyst,"Partner's Consulting, Inc.","Philadelphia, Pennsylvania"
147,Data Analyst,New York Life Insurance Company,"New York City, NY, US"
148,Data Analyst,24 Seven LLC,"Portland, Oregon"


## Challenge 2

Further improve your function so that it can search jobs in a specific country. Add the 3rd param to your function called `country`. The steps are identical to those in Challange 1.

In [103]:
# your code here

def scrape_linkedin_job_search_pages_country(keywords,num_pages,country):
    
    
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    if country==True:
        scrape_url = ''.join([BASE_URL, 'keywords=', keywords,])
    else:   
        scrape_url = ''.join([BASE_URL, 'keywords=', keywords,'&location=',country.replace(" ","%20")]) 
        

    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)
    
    for num in range(num_pages):
        if num == 0:
            page = requests.get(scrape_url)
        else:
            page = requests.get(''.join([scrape_url,'&start=',str(num*25)]))
        soup = BeautifulSoup(page.text, 'html.parser')
        titles = []
        companies = []
        locations = []
        for card in soup.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
        zipped = zip(titles, companies, locations)
        for z in list(zipped):
            data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    return data


In [113]:
results = scrape_linkedin_job_search_pages_country('data%20analysis',4,'UK')
results

,Title,Company,Location
0,Lead Data Analyst,EPAM Systems,"Kiev, UA"
1,Senior Data Analyst - Marketplace,thredUP,"Kiev, UA"
2,SalesForce PMO,Luxoft,"Kiev Region, Ukraine"
3,Senior Big Data Software Engineer (ID 40222),SoftServe,"Dnipropetrovsk, Ukraine"
4,Product Analyst,Toptal,"Kharkiv, UA"
...,...,...,...
95,DATA MIGRATION DEVELOPER,INSCALE Global,"Kiev, UA"
96,"Accounting and data analysis specialist, UA",JACOBS DOUWE EGBERTS,"Kiev, UA"
97,Data Science Engineer,Capital Recruiters,"Kiev, UA"
98,Company Analytics and Data Intelligence Lead,LifeCell International Pvt.Ltd,"Kiev, UA"


False

## Challenge 3

Add the 4th param called `num_days` to your function to allow it to search jobs posted in the past X days. Note that in the LinkedIn job search the searched timespan is specified with the following param:

```
f_TPR=r259200
```

The number part in the param value is the number of seconds. 259,200 seconds equal to 3 days. You need to convert `num_days` to number of seconds and supply that info to LinkedIn job search.

In [120]:
# your code here

def scrape_linkedin_job_search_pages_country_days(keywords,num_pages,country,num_days):
    
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    scrape_url = ''.join([BASE_URL, 'keywords=', keywords,'&location=',country.replace(" ","%20"),'f_TPR=r',str(num_days*86400)]) 
        
        
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)
    
    for num in range(num_pages):
        if num == 0:
            page = requests.get(scrape_url)
        else:
            page = requests.get(''.join([scrape_url,'&start=',str(num*25)]))
        soup = BeautifulSoup(page.text, 'html.parser')
        titles = []
        companies = []
        locations = []
        for card in soup.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
        zipped = zip(titles, companies, locations)
        for z in list(zipped):
            data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    return data



In [121]:
results = scrape_linkedin_job_search_pages_country_days('data%20analysis',4,'United Kingdom',2)
results

,Title,Company,Location
0,Sr. Data Engineer,Cypress HCM,Miami/Fort Lauderdale Area
1,Senior Data Engineer,"Dynamo, LLC.",Greater Omaha Area
2,Data Architect,SessionM,Greater Boston Area
3,Lead Strategic Data Analyst (SAS & Tableau) Lo...,Compass Technology Group,"Dallas, Texas"
4,IT Data Analyst,Navigant,"Gardena, California"
...,...,...,...
95,"Manager, Reporting and Data Analysis",The Madison Square Garden Company,Greater New York City Area
96,Data Analyst,Viper Staffing Services L.L.C.,"Glendale, California, United States"
97,Sr. Bioinformatics Scientist,The EAC Agency,"Fremont, California, United States"
98,Programmer Anaylyst,Valley First Credit Union,"Modesto, California Area"


## Bonus Challenge

Allow your function to also retrieve the "Seniority Level" of each job searched. Note that the Seniority Level info is not in the initial search results. You need to make a separate search request for each job card based on the `currentJobId` value which you can extract from the job card HTML.

After you obtain the Seniority Level info, update the function and add it to a new column of the returned dataframe.

In [132]:
import re

# your code here

def scrape_linkedin_job_search_advanced(keywords,num_pages,country,num_days):
    
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    scrape_url = ''.join([BASE_URL, 'keywords=', keywords,'&location=',country.replace(" ","%20"),'f_TPR=r',str(num_days*86400)]) 
        
        
    columns = ['Title', 'Company', 'Location','CurrentJobId']
    data = pd.DataFrame(columns=columns)
    
    for num in range(num_pages):
        if num == 0:
            page = requests.get(scrape_url)
        else:
            page = requests.get(''.join([scrape_url,'&start=',str(num*25)]))
        soup = BeautifulSoup(page.text, 'html.parser')
        titles = []
        companies = []
        locations = []
        jobID = []
        for card in soup.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            jobID = card.find('a',attrs={"href": "job-result-card__location"})
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)
            jobID.append(job_ID)
        zipped = zip(titles, companies, locations, jobID)
        for z in list(zipped):
            data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2] , 'CurrentJobId':z[3]}, ignore_index=True)
    
    return data

In [140]:
BASE_URL = 'https://www.linkedin.com/jobs/search/?&f_TPR=r86400&keywords=data%20analysis'
page = requests.get(BASE_URL)
soup = BeautifulSoup(page.text, 'html.parser')
soup.select("div.result-card__contents")


[<div class="result-card__contents job-result-card__contents"><h3 class="result-card__title job-result-card__title">Software Engineer - Data Mining/Data Analysis/Machine Learning</h3><h4 class="result-card__subtitle job-result-card__subtitle"><a class="result-card__subtitle-link job-result-card__subtitle-link" data-tracking-control-name="guest_job_search_job-result-card_result-card_subtitle-click" data-tracking-will-navigate="" href="https://www.linkedin.com/company/linkedin?trk=guest_job_search_job-result-card_result-card_subtitle-click">LinkedIn</a></h4><div class="result-card__meta job-result-card__meta"><span class="job-result-card__location">Sunnyvale</span><p class="job-result-card__snippet">The ideal candidate will have domain experience (data mining, information retrieval, security data science, natural language processing, ...</p><time class="job-result-card__listdate--new" datetime="2019-10-28">Just now</time></div></div>,
 <div class="result-card__contents job-result-card__c

In [133]:
scrape_linkedin_job_search_advanced('data%20analysis',4,'United Kingdom',2)

AttributeError: 'NoneType' object has no attribute 'append'